Absolute magnitude is defined to be the apparent magnitude an object would have if it were located at a distance of 10 parsecs.
In astronomy, absolute magnitude (M) is a measure of the luminosity of a celestial object on an inverse logarithmic astronomical magnitude scale.

In [1]:
import time
import numpy as np
import h5py
import hdf5plugin
import pandas as pd
import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz
#from astropy.cosmology import Planck18 as cosmo
from scipy import signal
import os 
import csv
import resource
import pickle
import shutil
import json
from synthesizer.grid import Grid
from synthesizer.parametric import SFH, ZDist
from synthesizer.particle.stars import sample_sfhz
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle.particles import CoordinateGenerator
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds

from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.kernel_functions import Kernel

from synthesizer.conversions import lnu_to_absolute_mag

current_directory = os.getcwd()
print("Current directory:", current_directory)


/tmp/ipykernel_3355/2576222646.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Current directory: /home/jovyan/camels/play/synth-play/LH


# Start the timer
start_time = time.time()


In [2]:

# Alternative method for LF:
# try this method again, but using AB mag instead of mass, and suply your own bins (up to -17, say)
def calc_lf(ab_mag, volume, massBinLimits):
# OG:        hist, dummy = np.histogram(np.log10(mstar), bins = massBinLimits)
        hist, dummy = np.histogram(ab_mag, bins = massBinLimits)
        hist = np.float64(hist)
        phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])
        phi_sigma = (np.sqrt(hist) / volume) /\
                    (massBinLimits[1] - massBinLimits[0]) # Poisson errors
        return phi, phi_sigma, hist
    


phi_arr =[] #phi
phi_sigma_arr =[] # phi_sigma
hist_arr = [] # hist
z_arr = [] #redshift_074, 
abs_mag_arr = [] #absolute mag (th filter)
Vcom_arr = [] # comoving vol
massBinLimits_arr = []


len(phi_arr)

In [3]:
grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5"
grid_dir = "/home/jovyan/"

# Create a new grid
#grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)

# instead of using a filter, which requires us to load in large SEDs first, pass the grid wavelength
#filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
#filt_lst = [filt1]

# Define a new set of wavelengths
lims_lams=(1400, 1600)

grid = Grid(grid_name, grid_dir=grid_dir, lam_lims=lims_lams, read_lines=False)
print(grid.shape)


(221, 7, 401)


In [4]:
# Define the directory where the text files will be saved
output_dir = "/home/jovyan/camels/play/synth-play/LH/output/"


In [5]:

# get gals
LH_X = 'LH_50'
dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/LH/' + LH_X
gals_074 = load_CAMELS_IllustrisTNG(
    dir_,
    snap_name='snapshot_074.hdf5', 
    fof_name='groups_074.hdf5',
    verbose=True,
)

/srv/conda/envs/notebook/lib/python3.9/site-packages/synthesizer/particle/galaxy.py:110: RuntimeWarning: In `load_stars`: one of either `initial_masses`, `ages` or `metallicities` is not provided, setting `stars` object to `None`
  self.load_stars(stars=stars)
/srv/conda/envs/notebook/lib/python3.9/site-packages/synthesizer/particle/galaxy.py:111: RuntimeWarning: In `load_gas`: one of either `masses` or `metallicities` is not provided, setting `gas` object to `None`
  self.load_gas(gas=gas)
/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1938: RuntimeWarning: invalid value encountered in divide
  out_arr = func(
/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1813: RuntimeWarning: invalid value encountered in log10
  out_arr = func(np.asarray(inp), out=out_func, **kwargs)



for gal in gals_074:
    print(f"Galaxy has {gal.stars.nstars} stellar particles")
    print(f"Galaxy gas {gal.gas.nparticles} gas particles")


gals_074

In [6]:

# Filter galaxies to only include those with 100 or more star particles
gals_074 = [gal for gal in gals_074 if gal.stars.nstars >= 100]
print(len(gals_074))
gals_074 = [gal for gal in gals_074 if gal.gas.nparticles >= 100]
print(len(gals_074))


571
507



for gal in gals_074:
    print(f"Galaxy has {gal.stars.nstars} stellar particles")
    print(f"Galaxy gas {gal.gas.nparticles} gas particles")


In [7]:


cat_074 = dir_+'/groups_074.hdf5'
# open file
f_h5py = h5py.File(cat_074, 'r')

# read different attributes of the header
boxSize_074 = f_h5py['Header'].attrs[u'BoxSize']/1e3 #Mpc/h
redshift_074 = f_h5py['Header'].attrs[u'Redshift']


In [8]:
# Get the number of star particles for each galaxy
nstars_list = [gal.stars.nstars for gal in gals_074]
# Get the number of gas particles for each galaxy
ngas_list = [gal.gas.nparticles for gal in gals_074]

# Calculate the minimum and maximum number of star particles
min_nstars = min(nstars_list)
max_nstars = max(nstars_list)

# Calculate the minimum and maximum number of gas particles
min_ngas = min(ngas_list)
max_ngas = max(ngas_list)

print(f"Minimum number of star particles: {min_nstars}")
print(f"Maximum number of star particles: {max_nstars}")
print(f"Minimum number of gas particles: {min_ngas}")
print(f"Maximum number of gas particles: {max_ngas}")


Minimum number of star particles: 100
Maximum number of star particles: 22781
Minimum number of gas particles: 101
Maximum number of gas particles: 192118


In [9]:
for gal in gals_074:
    print(gal.stars.ages)

[6.61692325e+09 6.69397668e+09 7.37412448e+09 ... 6.92768978e+09
 7.06748076e+09 3.29105000e+09] yr
[6.48868461e+09 6.48710578e+09 6.48967697e+09 ... 6.64359067e+09
 5.24730561e+09 6.37624972e+09] yr
[4.02450229e+09 4.69090922e+09 4.38597603e+09 ... 3.43276681e+08
 3.40052826e+08 6.91914217e+09] yr
[2.72143208e+08 6.12811172e+09 5.32514397e+09 9.51501514e+08
 6.81904587e+09 5.04522928e+09 7.90712497e+08 4.84902234e+09
 9.57628867e+07 5.42450144e+09 4.91290393e+09 7.12057577e+09
 3.81899545e+08 5.90938058e+09 5.56504764e+09 1.15961032e+07
 4.41999538e+09 6.17302423e+08 1.72439936e+09 1.20869133e+09
 2.48608391e+08 5.83457127e+09 5.53853180e+09 4.16103658e+09
 6.02107542e+09 6.73366760e+09 2.78626910e+08 5.34345472e+08
 4.36401122e+08 6.48653548e+09 4.07812687e+09 2.43742561e+09
 5.55733568e+08 5.94961043e+09 7.10822359e+09 4.45097298e+09
 1.21836962e+09 4.61289249e+09 4.70422719e+09 6.63112324e+09
 4.72192391e+09 4.01228707e+09 1.47364135e+09 3.86718776e+08
 4.67309149e+09 7.27918389e+0

[7.23948484e+09 8.05743902e+09 5.78414224e+09 ... 6.15425880e+09
 4.24265462e+09 7.19880505e+09] yr
[6.57153101e+09 5.11023936e+09 3.07969895e+09 7.29034947e+09
 6.69493009e+09 6.51039074e+09 7.35038602e+09 8.11808912e+09
 5.29911812e+09 3.23354796e+09 3.70863413e+09 5.62801099e+09
 2.69604818e+09 2.27623208e+09 7.72992578e+09 6.14240380e+09
 2.54819807e+09 3.21087071e+09 6.17547218e+09 6.33175742e+09
 3.87087010e+09 5.63221437e+09 6.37679433e+09 6.18952521e+09
 4.99412801e+09 3.88652992e+09 3.87713969e+09 6.15662346e+09
 4.36221284e+09 5.07948921e+09 3.13530420e+09 3.57486773e+09
 3.67736581e+09 6.04574986e+09 2.85512884e+09 4.21428331e+09
 3.78661183e+09 5.98116453e+09 3.77811757e+09 6.37352545e+09
 4.28540226e+09 4.87554156e+09 7.47608751e+09 3.25672529e+09
 3.35602539e+09 3.34405288e+09 4.44535555e+09 7.21052912e+09
 7.24950055e+09 3.46277026e+09 4.06199452e+09 2.14546246e+09
 6.22780730e+09 4.95161793e+09 2.59663945e+09 4.59476940e+09
 3.48624679e+09 7.12211394e+09 3.19746041e+09 

[ 3.34405288e+09  2.44650048e+09  3.17458840e+08  6.20699204e+09
  1.62433425e+09  3.64212600e+09  2.63340092e+09  3.18928466e+09
  5.84508696e+09  1.64286796e+09  6.75880780e+09  3.12941039e+09
  3.83731686e+09  5.53997010e+09  3.96212185e+09  1.19079957e+09
  7.48099263e+09  3.26828322e+09  3.64375739e+09  3.14001575e+09
  3.22774071e+09  2.19908972e+09  3.75469303e+09  5.55145636e+09
  2.65864896e+09  3.59791875e+09  1.64286796e+09  3.12233138e+09
  4.13022170e+09  6.15726920e+08  5.66565548e+09  3.73117334e+09
  3.56165614e+09  3.30972210e+09  6.30390891e+09  3.68384864e+09
  3.48736190e+09  1.68549707e+09  5.01893355e+09  3.59901373e+09
  3.57266805e+09  2.77493778e+09  4.44160648e+09  2.44650048e+09
  5.03867149e+09  2.35002543e+09  1.65425508e+09  6.61340597e+09
  3.14354681e+09  3.31201632e+09  4.72104010e+09  7.15410514e+09
  7.58012453e+09  5.95087878e+09  3.83100305e+09  6.38114653e+09
  3.63614216e+09  3.19395774e+09  3.69463777e+09  3.12587120e+09
  3.23819005e+09  5.59278

[4.97084233e+09 5.51108887e+09 4.21624488e+09 5.82930048e+09
 4.97084233e+09 5.34568508e+09 5.17500855e+09 4.11224705e+09
 4.60021384e+09 5.30218990e+09 6.09085616e+09 4.85759489e+09
 4.41999538e+09 4.34694477e+09 5.22544333e+09 5.93816993e+09
 2.95381498e+09 4.00820918e+09 5.04686759e+09 4.42846332e+09
 6.86883116e+09 6.80641068e+09 4.34407716e+09 5.51398749e+09
 4.63184556e+09 5.69464699e+09 5.97110267e+09 6.95311222e+09
 5.18214034e+09 3.41780786e+09 4.39923399e+09 6.16606380e+09
 5.53709287e+09 7.08011510e+09 5.41408980e+09 3.46277026e+09
 5.49510331e+09 7.09890272e+09 6.19769008e+09 4.31626746e+09
 5.59136325e+09 5.48489290e+09 6.19885473e+09 5.56576123e+09
 6.00118678e+09 6.43279851e+09 5.84771029e+09 5.73698667e+09
 4.34407716e+09 4.85245301e+09 5.56576123e+09 5.00902936e+09
 4.24460502e+09 4.14416000e+09 6.37025217e+09 5.49510331e+09
 4.39828818e+09 7.26934954e+09 5.48050889e+09 5.25508310e+09
 4.47151594e+09 6.14002685e+09 6.34612202e+09 7.34098855e+09
 5.87512528e+09 4.095214

[7.03563218e+08 4.58931902e+09 6.11002606e+08 2.38013682e+09
 3.73545645e+09 3.58146218e+09 5.12632966e+09 1.46457451e+09
 5.70836415e+09 4.24070291e+09 5.60268103e+09 1.35069100e+09
 5.03949184e+09 6.81002842e+09 3.09379086e+08 1.95778236e+09
 5.47172306e+09 5.68087273e+09 5.44966478e+09 4.22897753e+09
 5.47318867e+09 4.37553447e+09 5.24341067e+09 1.59745995e+08
 5.59702693e+09 4.25824069e+09 4.80590565e+09 4.68468118e+09
 5.26206992e+09 4.26795848e+09 5.22152676e+09 9.46154677e+08
 2.93685630e+09 5.76939563e+09 2.07794067e+09 2.91585400e+08
 2.06301264e+09 3.73117334e+09 6.33729143e+09 7.34766729e+08
 1.64001888e+09 1.76032566e+09 2.24174018e+09 2.75506227e+09
 4.84558850e+09 4.71131424e+09 1.59145317e+09 2.34215111e+09
 4.41716945e+09 2.25104111e+09 6.19769008e+09 2.70638622e+09
 5.09246518e+09 1.39172611e+09 1.65283219e+09 3.01644399e+09
 5.75728016e+09 2.06029673e+09 5.22544333e+09 4.19364089e+09
 1.87072209e+09 1.36243349e+09 5.76536204e+09 1.16689380e+09
 8.30941000e+08 4.627339

...

[2.94534050e+09 3.91875627e+09 8.27851923e+08 5.66704201e+09
 4.26018583e+09 2.13738817e+09 3.39294045e+09 2.27093546e+09
 4.35076606e+09 2.57500652e+09 1.54551260e+09 2.92957585e+09
 5.22152676e+09 5.42152991e+09 4.03364629e+09 4.00922898e+09
 1.22052252e+08 3.99084382e+09 2.70012240e+09 2.72389738e+09
 2.43353203e+09 4.67844436e+09 3.88131445e+09 2.72889237e+09
 5.54428166e+09 5.98618258e+09 5.26981800e+09 4.09220212e+09
 6.82443776e+09 4.01228707e+09 4.18575510e+09 5.44966478e+09
 2.71264479e+09 4.58022049e+09 4.19068520e+09 2.19641595e+09
 5.26826985e+09 5.31291919e+09 3.93531647e+09 4.53262251e+09
 1.89291487e+09 5.95341433e+09 3.54620570e+09 5.35099158e+09
 5.00241318e+09 5.33428723e+09 5.87642526e+09 4.65968230e+09
 2.81822833e+09 5.96858208e+09 5.99119215e+09 6.12930550e+09
 4.82665437e+09 5.70562520e+09 4.61831539e+09 4.70067971e+09
 7.94089052e+09 4.21330202e+09 6.06778076e+09 6.47504390e+09
 3.45492488e+09 5.69189684e+09 5.52555887e+09 5.73155417e+09
 5.59702693e+09 4.639944

[1.07976360e+09 7.01116342e+09 8.95612519e+08 5.89714420e+09
 2.28813755e+09 7.57099504e+09 5.97613800e+09 6.36588188e+09
 7.17957822e+09 7.01442783e+09 6.45401120e+09 5.18134836e+09
 7.21344763e+09 2.29474385e+09 5.93816993e+09 7.27778261e+09
 1.19229200e+09 3.18109860e+09 1.23249923e+09 5.06321095e+09
 7.50405874e+09 9.35453074e+08 5.16150220e+09 5.61818254e+09
 6.04206182e+09 7.45753946e+09 4.93062143e+09 5.52844574e+09
 6.43811842e+09 9.97217265e+08 1.05559722e+09 1.13843231e+09
 7.38610047e+09 7.50335848e+08 6.38549077e+09 3.06899892e+09
 6.49901089e+09 6.26555970e+09 3.86877869e+09 6.67193617e+09
 6.75880780e+09 6.84941588e+09 4.39734206e+09 1.39465085e+09
 5.93307151e+09 4.30326936e+09 4.63994469e+09 6.59953108e+09
 4.22506359e+09 1.09182487e+09 5.72202501e+09 7.68272487e+09
 4.48268393e+09 3.72903044e+09 8.27079767e+08 2.63213669e+09
 6.78822802e+09 3.90838116e+09 2.23243011e+09 1.12191828e+09
 6.46665216e+09 6.64773138e+09 9.84284144e+08 6.21973097e+09
 1.04803345e+09 6.664218

[6.27463248e+09 6.38549077e+09 4.24265462e+09 3.98367715e+09
 5.61677659e+09 2.96590250e+09 5.07786441e+09 3.63178558e+09
 4.99246882e+09 5.96479757e+09 5.84508696e+09 4.86101900e+09
 4.22114569e+09 5.92924234e+09 4.57839869e+09 5.71383506e+09
 5.44523696e+09 4.69535299e+09 2.83916204e+09 5.00241318e+09
 6.46665216e+09 7.25092642e+09 5.21839037e+09 3.47284285e+09
 4.05188708e+09 5.39766808e+09 6.39091160e+09 6.14952264e+09
 7.91766066e+09 6.20815272e+09 5.49364673e+09 5.11023936e+09
 5.24652645e+09 5.32209183e+09 3.76960998e+09 3.23238672e+09
 3.12941039e+09 4.11024529e+09 4.63634631e+09 3.73973701e+09
 5.17580136e+09 5.27445906e+09 2.79723366e+09 4.18969931e+09
 2.86861116e+09 6.84408881e+09 5.40364835e+09 7.44378183e+09
 4.99412801e+09 5.93562207e+09 4.00106526e+09 4.45284394e+09
 6.37570517e+09 5.12954013e+09 5.17421500e+09 7.11286571e+09
 6.97654374e+09 7.49318053e+09 6.28367514e+09 6.82623193e+09
 5.74647267e+09 6.61735475e+09 5.25586016e+09 6.27236704e+09
 7.40558039e+09 5.305258

[-1.68397285e+02  5.36611279e+09  5.94580184e+09  7.08326020e+09
  5.77610693e+09  6.23356090e+09  6.64285913e+09  7.02580862e+09
  5.31138811e+09  7.55254255e+09  4.68343158e+08  2.45297726e+09
  2.53283591e+09  2.36182151e+09  1.71945861e+09  2.25635084e+09
  3.90422541e+09  2.20843812e+09  4.67576845e+09  3.13765981e+09
  4.59658468e+09  3.05470602e+09  3.20688582e+08  1.76099065e+08
  2.79970690e+09  6.83233539e+08  6.91057438e+08  2.69510859e+09
  2.78980909e+09  7.36324945e+08  2.75385538e+08  1.68266113e+09
  6.95647686e+09  2.65738900e+09  1.37416076e+09  1.56276559e+09
  3.45043750e+09  2.85390151e+09  3.83731686e+09  3.43695622e+09
  4.07208283e+09  7.05475858e+09  3.96623402e+09  3.99493492e+09
  6.64870455e+09  4.49382693e+09  3.27290091e+09  1.60576332e+09
  3.66871102e+09  7.45629423e+09  3.89174044e+09  6.14002685e+09
  7.14198654e+09  5.61677659e+09  2.72514621e+09  2.38078996e+09
  3.01884219e+09  1.51668513e+09  1.68124351e+09  5.28141097e+09
  1.81818926e+08  7.28533

[4.71839018e+09 2.14815235e+09 5.82666130e+09 5.34112963e+09
 3.29823834e+09 6.23598281e+08 5.24029186e+09 3.81096405e+09
 5.70082653e+09 5.74274930e+09 2.38274991e+09 5.35402075e+09
 5.78414224e+09 1.22209045e+09 5.49437535e+09 3.07969895e+09
 5.38942951e+09 5.08476585e+09 4.46125513e+09 1.50512893e+09
 4.51049235e+09 5.63571335e+09 1.77016287e+09 3.27059282e+09
 4.26018583e+09 2.34608869e+09 6.83339299e+09 5.35629101e+09
 2.53411803e+09 4.43410041e+09 1.08914218e+08 9.40040392e+08
 1.23844180e+09 5.06157964e+09 5.93944329e+09 6.56593761e+08
 2.26430947e+09 1.02986004e+09 5.40514183e+09 2.71764783e+09
 1.08579554e+09 1.54695097e+09 6.06167796e+09 1.93023692e+09
 3.99288969e+09 4.88491264e+09 3.22541681e+09 4.16301787e+09
 1.64654871e+08 4.26310172e+09 3.20096150e+09 3.85112537e+08
 8.71022252e+08 1.32517808e+07 1.45150128e+09 4.44722848e+09
 2.68506814e+09 4.50031476e+09 4.05997433e+09 6.27802726e+09
 5.63781095e+09 1.55701833e+09 5.84771029e+09 6.00617155e+09
 4.52894095e+09 5.935622

In [ ]:
# FAILS HERE
spec_list = []
# Lets work with z=0 so gals_025
for gal in gals_074:
    # get_spectra_incident An Sed object containing the stellar spectra
    spec = gal.stars.get_spectra_incident(grid)
    spec.get_fnu0()
    spec_list.append(spec)


In [ ]:

# combine
seds = combine_list_of_seds(spec_list)

seds.lnu # rest frame lumd
seds.get_photo_luminosities(filt_lst)
seds.photo_luminosities.photo_luminosities
abs_mag = lnu_to_absolute_mag(seds.photo_luminosities.photo_luminosities)
abs_mag_th = abs_mag[0] # taking first argument which is the tophat filter.


In [ ]:
# next steps, get luminosity function for these magnitudes
# co-moving volume: BoxSize_025 and redshift:
little_h =  0.6711
Vphys = (boxSize_074/little_h )**3
Vcom = Vphys * ((1+redshift_074)**3)


In [ ]:

massBinLimits = np.arange(-22, -16, 0.5)
phi, phi_sigma, hist = calc_lf(abs_mag_th, Vcom, massBinLimits)
massBinLimits = massBinLimits[:-1]


In [ ]:
output_dir = "/home/jovyan/camels/play/synth-play/LH/output/"

# Define output file path
output_file = f"{output_dir}{LH_X}_FIX.txt"

# Write the data to the text file line by line
with open(output_file, 'w') as txtfile:
    # Write phi values
    txtfile.write("phi\n")
    for value in phi:
        txtfile.write(f"{value}\n")

    # Write phi_sigma values
    txtfile.write("phi_sigma\n")
    for value in phi_sigma:
        txtfile.write(f"{value}\n")

    # Write hist values
    txtfile.write("hist\n")
    for value in hist:
        txtfile.write(f"{value}\n")

    # Write massBinLimits values
    txtfile.write("massBinLimits\n")
    for value in massBinLimits:
        txtfile.write(f"{value}\n")


print('Written out: /home/jovyan/camels/play/synth-play/LH/output/',LH_X)

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
# Get memory usage in bytes
memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
print("Memory usage end:", memory_usage/1000000, "GB")


# will do latest LH_ set in loop as a test
label_z = 'z = 0.46'


# Plot the luminosity function
plt.errorbar(massBinLimits, phi, yerr=phi_sigma, fmt='o', color='blue',label=label_z)
plt.xlabel('Absolute Magnitude (AB)')
plt.ylabel('Number Density (Mpc^-3 mag^-1)')
plt.yscale('log')

plt.title('Luminosity Function XMM-OM filter')
plt.grid(True)
plt.show()